# CME 257 HW 3
## Ren Gibbons 

This notebook contains a simple example of a finite element assembly procedure for a 1D truss simulation using the Julia package JuliaFEM. The example is slightly modified from a tutorial page on JuliaFEM found [here](http://www.juliafem.org/examples/2017-08-30-defining-new-problems-to-juliafem/). This notebook was greated using Julia 0.6.2 version and the package JuliaFEM. Here, I import JuliaFEM for use in this notebook and check the status.

In [9]:
using JuliaFEM
import JuliaFEM: assemble!
Pkg.status("JuliaFEM")

 - JuliaFEM                      0.3.6


Next, I define the element type Truss as a descendant of FieldProblem, where FieldProblem describes a general framework for a finite element problem to be solved.

In [16]:
type Truss <: FieldProblem
end

Here, I define the assembly procecedure. First, the element stiffness matrices are created given geometrc and material properties. Next, I assign the values of the element stiffness matrix to the appropriate degrees of freedom in the global stiffness matrix.

In [11]:
function assemble!(assembly::Assembly, problem::Problem{Truss},
                   element::Element{Seg2}, time)
    X = element("geometry", time)        # get geometry
    L = norm(X[2] - X[1])                # calculate length of rod
    E = 1.0                              # Young's modulus
    A = 1.0                              # cross sectional area of element
    q = 1.0                              # magnitude of force
    Ke = E*A/L*[1.0 -1.0; -1.0 1.0]      # element stiffness matrix 
    fe = q*L/2*[1.0, 1.0]                # element force vector
    gdofs = get_gdofs(problem, element)  # find global dofs of element
    add!(assembly.K, gdofs, gdofs, Ke)   # element -> global 
    add!(assembly.f, gdofs, fe)          # force   -> global
end

assemble! (generic function with 37 methods)

Finally, I test the element by defining a 1D structure with three truss elements with nodes located at positions $$x=[0.0,1.0,2.0,3.0]$$

In [18]:
X = Dict(1 => [0.0], 2 => [1.0], 3 => [2.0], 4 => [3.0]) # Location of nodes (or Truss element end points)
element1 = Element(Seg2, [1, 2])             # First 1D element (with ends at node 1 and 2)
element2 = Element(Seg2, [2, 3])             # Second 1D element (with ends at node 2 and 3)
element3 = Element(Seg2, [3, 4])             # Third 1D element (with ends at node 3 and 4)
elements = [element1, element2, element3]
update!(elements, "geometry", X)             # Create the mesh
problem = Problem(Truss, "test problem", 1)  # Initialize problem
add_elements!(problem, elements)             # Add elements to problem
assemble!(problem)                           # Assemble stiffness matrix and force vector for Truss

04-Feb 18:03:09:WARNING:root:assemble!(problem) will be deprecated. Use assemble!(problem, time)
04-Feb 18:03:09:WARNING:root:Assembling elements for problem test problem: seems that problem is uninitialized. To initialize problem, use `initialize!(problem, time)`.
04-Feb 18:03:09:INFO:root:Initializing problem test problem at time 0.0 automatically.
04-Feb 18:03:09:WARNING:root:This is default assemble! function. Decreased performance can be expected without preallocation of memory. One should implement `assemble_elements!(problem, assembly, elements, time)` function.


I verify the correctness of the element implementation by printing the stiffness matrix and force vector.

In [19]:
full(problem.assembly.K)

4×4 Array{Float64,2}:
  1.0  -1.0   0.0   0.0
 -1.0   2.0  -1.0   0.0
  0.0  -1.0   2.0  -1.0
  0.0   0.0  -1.0   1.0

In [20]:
full(problem.assembly.f)

4×1 Array{Float64,2}:
 0.5
 1.0
 1.0
 0.5

This example only shows how to create a new element type. This [page](http://www.juliafem.org/JuliaFEM.jl/v0.3.2/examples.html) provides an insightful tutorial for setting up and solving a simple finite element problem for a single element 2D quad elastic element.